In [1]:
import torch
import gradio as gr
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler, AutoPipelineForInpainting, StableDiffusionPipeline

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [2]:
imageWidth = 512    
imageHeight = 512                                         
guidance_scale = 8                                                             
model_path = "segmind/tiny-sd" 

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(torch_device)
scheduler = LMSDiscreteScheduler.from_pretrained(model_path, subfolder="scheduler")
unet = UNet2DConditionModel.from_pretrained(model_path, subfolder="unet").to(torch_device)
vae = AutoencoderKL.from_pretrained(model_path, subfolder="vae").to(torch_device)

In [3]:
pipe = StableDiffusionPipeline(
    tokenizer=tokenizer,
    text_encoder=text_encoder,
    unet=unet,
    scheduler=scheduler,
    vae=vae,  
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False,
    ).to(torch_device)

In [4]:
inpaint_pipe = AutoPipelineForInpainting.from_pipe(pipe).to(torch_device)

vae\diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
def predict(dict, seed, prompt=""):
    mask = dict["layers"][0].convert("RGB").resize((imageHeight, imageWidth))
    init_image = dict["background"].convert("RGB").resize((imageHeight, imageWidth))
    output = inpaint_pipe(prompt = prompt, image=init_image, mask_image=mask, guidance_scale=guidance_scale, generator=torch.Generator().manual_seed(seed))
    return output.images[0]

In [9]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            image = gr.Sketchpad(sources='upload', type="pil", label="Upload", brush=gr.Brush(colors=["#ffff00"]))
        with gr.Column():
            image_out = gr.Image(label="Output")

    with gr.Row():
        prompt = gr.Textbox(label="Prompt")
        seed = gr.Number(label="Seed", value=0)
        btn = gr.Button("Inpaint")

    btn.click(fn=predict, inputs=[image, seed, prompt], outputs=[image_out])

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


  0%|          | 0/50 [00:00<?, ?it/s]